<a href="https://colab.research.google.com/github/TiagoSpiandorello/Webscraping_icarros/blob/main/icarros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
import pandas as pd

cards = []

quantidade_paginas = 5

for i in range(quantidade_paginas):

    url = 'https://www.icarros.com.br/ache/listaanuncios.jsp?bid=1&app=20&sop=nta_17|44|51.1_-est_SC.1_-cid_8452.1_-rai_50.1_-esc_2.1_-sta_1.1_&pas=1&lis=0&pag='+ str(i + 1) + '&ord=14'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

    try:
        req = Request(url, headers= headers)
        response = urlopen(req)
        html = response.read()
        
    except HTTPError as e:
        print(e.status, e.reason)

    except URLerror as e:
        print(e.reason)

    soup = BeautifulSoup(html, 'lxml')

    items = soup.findAll('div', {'class': 'anuncio_container'})

        
    for item in items:
        card = {}

        if (item.find('div', {'class': 'feirao-tag'})):
          card['titulo'] = item.find('a', {'class': 'clearfix'}).get('title')
          card['url'] =  'https://www.icarros.com.br' + item.find('a', {'class': 'clearfix'}).attrs['href']
        else:
          card['titulo'] = item.find('a').get('title')
          card['url'] =  'https://www.icarros.com.br' + item.find('a').attrs['href']

        card['ano'] =  item.find('li', {'class': 'primeiro'}).p.get_text().replace('\n','').replace(' ','').split('/')[0]
        card['modelo'] =  item.find('li', {'class': 'primeiro'}).p.get_text().replace('\n','').replace(' ','').split('/')[-1]

        if (item.find('li', {'class': 'usado'})):
          card['km'] = item.find('li', {'class': 'usado'}).p.get_text().replace('\n','').replace(' ','').replace('km','')
        else:
          card['km'] =  item.find('li', {'class': 'zerokm'}).get_text().replace('\n','').replace(' ','').replace('km','')

        card['cambio'] = item.find('li', {'class': 'ultimo'}).p.get_text().replace('\n','')
        card['preco'] = item.find('h3', {'class': 'preco_anuncio'}).get_text().replace('R$ ','').replace('preço à vista','')

        cards.append(card)

        #print(str(card) + '\n')
  
dataset = pd.DataFrame(cards)
dataset.url = dataset.url.str.replace('®','&reg')
dataset = dataset[['titulo','ano','modelo','km','cambio','preco','url']]

print('A pesquisa foi concluída com sucesso. Foram encontrados ' + str(len(dataset)) + ' anúncios')

dataset.to_csv('anuncios_icarros.csv', sep=';', index=False, encoding='utf-8')

A pesquisa foi concluída com sucesso. Foram encontrados 100 anúncios


In [ ]:
dataset

,titulo,ano,modelo,km,cambio,preco,url
0,Peugeot 208 1.6 Active (Aut),2020,2021,0,automático,"82.180,00",https://www.icarros.com.br/comprar/peugeot/208...
1,Peugeot 208 1.6 Active Pack (Aut),2020,2021,0,automático,"88.980,00",https://www.icarros.com.br/comprar/peugeot/208...
2,Peugeot 208 1.6 Allure (Aut),2020,2021,0,automático,"93.880,00",https://www.icarros.com.br/comprar/peugeot/208...
3,Peugeot 2008 1.6 Allure Pack (Aut),2021,2021,0,automático,"96.990,00",https://www.icarros.com.br/comprar/peugeot/200...
4,Peugeot 208 1.6 Griffe (Aut),2020,2021,0,automático,"98.880,00",https://www.icarros.com.br/comprar/peugeot/208...
...,...,...,...,...,...,...,...
95,Renault Logan Expression 1.6 16V SCe (Flex),2019,2020,42.074,manual,"46.380,00",https://www.icarros.com.br/comprar/florianopol...
96,Chevrolet Joy Plus 1.0 SPE/4 Eco,2019,2020,41.504,manual,"46.450,00",https://www.icarros.com.br/comprar/sao-jose-sc...
97,Peugeot 208 1.2 Active,2019,2020,35.200,manual,"46.500,00",https://www.icarros.com.br/comprar/florianopol...
98,Ford Ka Sedan SE 1.0 (Flex),2019,2020,48.855,manual,"46.500,00",https://www.icarros.com.br/comprar/florianopol...
